<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_opsmps3copy.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Agnostic App  📒
***Create your own Tapis App***

by Silvia Mazzoni, DesignSafe, 2025

This is a step-by-step guide to help you write your own **Tapis v3 App** — from defining the app, to registering it, and running it. 

This is a practical walkthrough for defining and deploying an **HPC** app using the Tapis v3 API.

In this notebook we will modify the opensees-mp-s3 for OpenSeesMP or OpenSeesSP, etc.

## Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [ ]:
pwd

In [ ]:
import json

In [ ]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

---
## Connect to Tapis

In [ ]:
t=OpsUtils.connect_tapis()

---
### Get username

In [ ]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['get_tapis_username.py'])

In [ ]:
username = OpsUtils.get_tapis_username(t)
print('username',username)

## Define actions!

In [ ]:
# Make these True once you are done validating
do_makeApp = True
do_submitJob = True

---
## Configure App

In [ ]:
app_id = 'agnostic-app-test'
app_description = 'Run any Executable'

### AutoIncrement app_version

In [ ]:
if do_makeApp:
    latest_app_version = OpsUtils.get_latest_app_version(t,app_id)
    print('now latest_app_version',latest_app_version)
    app_version = OpsUtils.bump_app_version(latest_app_version,'patch')
    print('now app_version',app_version)

### Where to store the files

In [ ]:
if do_makeApp:
    app_system_id="designsafe.storage.default"
    app_path = f"{username}/apps/{app_id}/{app_version}"
    container_filename = f'{app_id}.zip'

### Define folder structure: app_id/version

In [ ]:
if do_makeApp:
    app_folderName = f'~/MyData/myAuthoredTapisApps/{app_id}/{app_version}'; # your choice

In [ ]:
if do_makeApp:
    app_folder = os.path.abspath(os.path.expanduser(app_folderName))
    os.makedirs(app_folder, exist_ok=True)
    print(f'app_folder: {app_folder}\n exists:',os.path.exists(app_folder))

---
## Create the App Files

A Tapis app needs the following **core files**:

* **Readme.MD** – App Description (OPTIONAL)<br>
This file is helpful in communicating content to the app user.  

* **app.json** – App Definition<br>
Defines the app’s metadata, inputs, parameters, and execution configuration.    

* **profile.json** – Environment Setup *(optional but common)*<br>
Loads modules or sets environment variables on the execution system. This file will load modules before your script runs. This is executed on the compute node.

* **tapisjob_app.sh** – Wrapper Script<br>
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [ ]:
# let's collect the list of files we are creating as we will need this list in upload
if do_makeApp:
    appFilesList = [];

#### **Readme.MD** – App Description
This file is helpful in communicating content to the app user.  

In [ ]:
if do_makeApp:
    thisFilename = 'ReadMe.MD'
    thisText = """\
    # making an application-agnostic app
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **app.json** – App Definition
Defines the app’s metadata, inputs, parameters, and execution configuration.    

In [ ]:
if do_makeApp:
    thisFilename = 'app.json'
    thisText = """\
    {
      "id": "__app_id__",
      "version": "__app_version__",
      "description": "__app_description__",
      "owner": "${apiUserId}",
      "enabled": true,
      "runtime": "ZIP",
      "runtimeVersion": null,
      "runtimeOptions": null,
      "containerImage": "__container_filename_path__",
      "jobType": "BATCH",
      "maxJobs": -1,
      "maxJobsPerUser": -1,
      "strictFileInputs": true,
      "jobAttributes": {
        "execSystemConstraints": null,
        "execSystemId": "stampede3",
        "execSystemExecDir": "${JobWorkingDir}",
        "execSystemInputDir": "${JobWorkingDir}",
        "execSystemOutputDir": "${JobWorkingDir}",
        "execSystemLogicalQueue": "skx",
        "archiveSystemId": "stampede3",
        "archiveSystemDir": "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
        "archiveOnAppError": true,
        "isMpi": false,
        "mpiCmd": null,
        "parameterSet": {
          "appArgs": [
            {
              "name": "Main Program",
              "description": "Choose the binary Executable to use.",
              "arg": "OpenSeesMP",
              "inputMode": "REQUIRED",
                "notes": {
                    "enum_values": [
                        {
                            "OpenSeesMP": "OpenSeesMP"
                        },
                        {
                            "OpenSeesSP": "OpenSeesSP"
                        },
                        {
                            "python": "python"
                        }
                    ]
                }
            },
            {
              "name": "Main Script",
              "description": "The filename only of the input script to submit to executable. This file should reside in the Input Directory specified.",
              "arg": null,
              "inputMode": "REQUIRED",
              "notes": {
                "inputType": "fileInput"
              }
            },
            {
              "name": "CommandLine Arguments",
              "description": "Command-Line Arguments (Optional)",
              "arg": null,
              "inputMode": "INCLUDE_ON_DEMAND",
              "notes": {
              }
            }
            
          ],
          "containerArgs": [],
          "schedulerOptions": [
            {
              "name": "OpenSees TACC Scheduler Profile",
              "description": "Scheduler profile for the default version of OpenSees",
              "inputMode": "FIXED",
              "arg": "--tapis-profile OpenSees_default",
              "notes": {
                "isHidden": true
              }
            }
          ],
          "envVariables": [
                  { "key": "zipFileIn", "value": "", "inputMode": "INCLUDE_BY_DEFAULT" },
                  { "key": "zipFolderOut", "value": "False", "inputMode": "INCLUDE_BY_DEFAULT" },
                  { "key": "requirementFile", "value": "", "inputMode": "INCLUDE_BY_DEFAULT" }
             ],
          "archiveFilter": {
            "includes": [],
            "excludes": [__container_filename__],
            "includeLaunchFiles": true
          }
        },
        "fileInputs": [
          {
            "name": "Input Directory",
            "inputMode": "REQUIRED",
            "sourceUrl": null,
            "targetPath": "inputDirectory",
            "envKey": "inputDirectory",
            "description": "Input directory that includes the input script as well as any other required files. Example input is in tapis://designsafe.storage.community/app_examples/opensees/OpenSeesMP",
            "notes": {
              "selectionMode": "directory"
            }
          }
        ],
        "fileInputArrays": [],
        "nodeCount": 1,
        "coresPerNode": 48,
        "memoryMB": 192000,
        "maxMinutes": 120,
        "subscriptions": [],
        "tags": []
      },
      "tags": [
        "portalName: DesignSafe",
        "portalName: CEP"
      ],
      "notes": {
        "label": "__app_id__",
        "helpUrl": "",
        "hideNodeCountAndCoresPerNode": false,
        "isInteractive": false,
        "icon": "OpenSees",
        "category": "Simulation"
      }
    }
    """
    thisText = thisText.replace("__app_id__", app_id)
    thisText = thisText.replace("__app_version__", app_version)
    thisText = thisText.replace("__app_description__", app_description)
    thisText = thisText.replace("__container_filename_path__", f"tapis://{app_system_id}/{app_path}/{container_filename}")
    thisText = thisText.replace("__container_filename__", container_filename)

    
    
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **profile.json** – Environment Setup
This file will load modules before your script runs. This is executed on the compute node.

In [ ]:
if do_makeApp:
    thisFilename = 'profile.json'
    thisText = """\
    {
        "name": "__app_id__",
        "description": "Modules to load",
        "moduleLoads": [ ],
        "hiddenOptions": [
            "MEM"
        ]
    }
    """    
    # thisText = """\
    # {
    #     "name": "AgnosticApp",
    #     "description": "Modules to load",
    #     "moduleLoads": [
    #         {
    #             "modulesToLoad": [
    #                 "hdf5/1.14.4",
    #                 "opensees"
    #             ],
    #             "moduleLoadCommand": "module load"
    #         }
    #     ],
    #     "hiddenOptions": [
    #         "MEM"
    #     ]
    # }
    # """
    thisText = thisText.replace("__app_id__", app_id)
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **tapisjob_app.sh** – Wrapper Script
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [ ]:
if do_makeApp:
    import textwrap, time
    from zipfile import ZipFile, ZIP_DEFLATED, ZipInfo
    
    thisFilename_sh = "tapisjob_app.sh"
    thisFilename = container_filename  # existing var


    bash_script = textwrap.dedent("""\
        #!/bin/bash
        set -euo pipefail
        set -x

        # ---- required args ----
        BINARYNAME="${1:?missing binary name}"
        INPUTSCRIPT0="${2:?missing input script}"
        shift 2

        # ---- optional env with safe defaults ----
        inputDirectory="${inputDirectory:?inputDirectory not set}"
        zipFileIn="${zipFileIn:-}"
        zipFolderOut="${zipFolderOut:-}"
        echo "zipFileIn: ${zipFileIn}"
        echo "zipFolderOut: ${zipFolderOut}"

        INPUTSCRIPT="${INPUTSCRIPT0##*/}"
        echo "INPUTSCRIPT: $INPUTSCRIPT"
        echo "ARGS: $*"

        cd -- "$inputDirectory"

        # ---- expand input ZIP if provided ----
        if [[ -n "$zipFileIn" && -f "$zipFileIn" ]]; then
          echo "Unzipping $zipFileIn ..."
          unzip -o -q "$zipFileIn"
        fi

        # ---- modules for OpenSees family ----
        case "$BINARYNAME" in
          OpenSees|OpenSeesMP|OpenSeesSP)
            module load hdf5/1.14.4
            module load opensees
            module list || true
            echo "Loaded OpenSees modules"
            ;;
        esac

        # ---- modules for python ----
        if [[ -n "$requirementFile" && -f "$requirementFile" ]]; then
          echo "Install requirements file"
          python -m pip install -r "$requirementFile"
        fi

        # ---- choose launcher ----
        if [[ "$BINARYNAME" == "OpenSees" ]]; then
          LAUNCH=()        # direct run for sequential
        else
          LAUNCH=(ibrun)   # MPI/SP use ibrun on TACC
        fi

        echo "Running: ${LAUNCH[*]} $BINARYNAME $INPUTSCRIPT $*"
        if ! "${LAUNCH[@]}" "$BINARYNAME" "$INPUTSCRIPT" "$@"; then
          rc=$?
          echo "Program exited with error status: $rc" >&2
          exit "$rc"
        fi
        echo "done, now zippit!"
        # ---- optional re-pack an output folder ----
        if [[ "${zipFolderOut:-}" =~ ^([Tt][Rr][Uu][Ee]|1|[Yy][Ee]?[Ss]?)$ ]]; then
          cd ..
          echo "Zipping output folder "$inputDirectory" -> ${zipFolderOut}.zip"
          zip -r -q "inputDirectory.zip" "$inputDirectory"
          rm -rf "$inputDirectory"
        fi
    """)


    zip_path = os.path.join(app_folder, thisFilename)
    print('zip_path',zip_path)

    # write with executable permission inside the ZIP
    zi = ZipInfo(thisFilename_sh)
    zi.date_time = time.localtime(time.time())[:6]
    zi.compress_type = ZIP_DEFLATED
    zi.external_attr = 0o100755 << 16   # -rwxr-xr-x on a regular file
    
    with ZipFile(zip_path, "w", ZIP_DEFLATED) as z:
        z.writestr(zi, bash_script)
        
    appFilesList.append(thisFilename)

#### File Check
Look at the files we have written and check for typos or formatting errors.

In [ ]:
if do_makeApp:
    print(appFilesList)
    OpsUtils.show_text_file_in_accordion(app_folder, appFilesList)

---
## Validate App Locally

In [ ]:
if do_makeApp:
    OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['validate_app_folder.py'])

In [ ]:
if do_makeApp:
    validation = OpsUtils.validate_app_folder(app_folder,appFilesList)
    if not validation:
        print('Validation Failed: stopping here!!!!')
        a = 3/0

---
## Deploy the App

### Make the directory inside your 'MyData' on designsafe.storage.default.
The apps in this folder are the ones that area actually uploaded.

In [ ]:
if do_makeApp:
    t.files.mkdir(systemId=app_system_id, path=app_path)
    print('app_path',app_path)

### Upload files to your deployment system (e.g., DesignSafe default storage)
Using Tapipy (Python SDK) in a Jupyter Notebook

In [ ]:
if do_makeApp:
    for fname in appFilesList:
        fpath = f'{app_folder}/{fname}'
        t.upload(source_file_path=fpath,
                 system_id=app_system_id,
                 dest_file_path=f'{app_path}/{fname}')

#### Verify upload

In [ ]:
if do_makeApp:
    print('app_system_id:',app_system_id)
    print('app_path:',app_path)
    appfiles = t.files.listFiles(systemId=app_system_id, path=app_path)
    for thisF in appfiles:
        print(thisF)
        print('')

---
## Register the App
This creates the actual App record that Jobs can run.

Do this Using Tapipy (Python)

In [ ]:
if do_makeApp:
    # Create (or create a new version) of the app
    with open(f'{app_folder}/app.json') as f:
        app_def = json.load(f)
    t.apps.createAppVersion(**app_def)

### Check that app is up

#### List all apps

In [ ]:
listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
select = 'id,created,description,version,owner' # Attributes to return in each result.
orderBy = 'created(asc)'
results = t.apps.getApps( orderBy=orderBy,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

#### List the new app

In [ ]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
print(appMetaData)

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

---
### Initialize

In [ ]:
tapisInputAll = {}

---
### SLURM-Specific Input

In [ ]:
tapisInputAll["maxMinutes"] = 6

tapisInputAll["execSystemId"] = "stampede3"
tapisInputAll["execSystemLogicalQueue"] = "skx-dev"
tapisInputAll["nodeCount"] = 1
tapisInputAll["coresPerNode"] = 48
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='Work' # Options: MyData or Work

---
### App-Specific Input

In [ ]:
print('app_id:',app_id)

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'OpenSees'
tapisInput['input_filename'] = 'Ex1a.Canti2D.Push.tcl'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in ['opensees-express',app_id]:
        print(f'\n -- {tapisInput['main_program']} {hereApp} --\n')
        here_tapisInput = tapisInput.copy()
        here_tapisInput["appId"] = hereApp 
        here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"]
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------